In [1]:
import os
from copy import deepcopy as dc

# data
import numpy as np
import tensorflow_datasets as tfds
import data.movi_a
import data.movi_b

# ml
import torch as pt

# physics
import warp as wp
import warp.sim as wps

# rendering
import drjit as dr
import mitsuba as mi
from mitsuba.scalar_rgb import Transform4f as mit

# point cloud
import pytorch3d as pt3d
from pytorch3d.structures import Meshes
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import (
    chamfer_distance,
)

# our stuff
from sssa.config import cfg

# setup
wp.init()
mi.set_variant('llvm_ad_rgb')
print("All packages imported!")

2024-02-23 19:10:55.668862: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Warp 0.10.1 initialized:
   CUDA Toolkit: 11.5, Driver: 11.4
   Devices:
     "cpu"    | x86_64
     "cuda:0" | NVIDIA GeForce GTX 1050 Ti with Max-Q Design (sm_61)
   Kernel cache: /home/jonathan/.cache/warp/0.10.1
All packages imported!


### Config

In [2]:
# fill in your directory set up here
project_dp = '/home/jonathan/git/sssa-object-modelling/'
config_fp = os.path.join(project_dp, f'cfg/templates/sssa-movi_a.yaml')
cfg.merge_from_file(config_fp)
cfg.DATA.path = os.path.join(project_dp, 'data/sets/')
print(cfg)

DATA:
  fps: 12
  fpv: 24
  height: 256
  max_objs: 10
  path: /home/jonathan/git/sssa-object-modelling/data/sets/
  set: movi_a
  width: 256
MODEL:
  DECODING:
    BASE:
      activation: relu
      arch: mlp
      batch_norm: True
      category_est: gumbel_softmax
      dropout: 0.1
      hidden_size: 2048
      num_layers: 1
    PARAMETERS:
      ANGULAR_VELOCITY:
        continuous: True
        include: True
        num_category: -1
      COLOUR:
        continuous: False
        include: True
        num_category: 9
      LINEAR_VELOCITY:
        continuous: True
        include: True
        num_category: -1
      MATERIAL:
        continuous: False
        include: True
        num_category: 2
      ORIENTATION:
        continuous: True
        include: True
        num_category: -1
      POSITION:
        continuous: True
        include: True
        num_category: -1
      SCALE:
        continuous: True
        include: True
        num_category: -1
  ENCODING:
    INTERACT

### Data

In [3]:
train_ds = tfds.load(cfg.DATA.set, 
                        data_dir=cfg.DATA.path,
                        split='train[:19]', 
                        shuffle_files=True)
train_ds = train_ds.shuffle(cfg.TRAIN.GPU.shuffle_buffer_size, reshuffle_each_iteration=True)
train_ds = tfds.as_numpy(train_ds)

test_ds = tfds.load(cfg.DATA.set, 
                   data_dir=cfg.DATA.path,
                   split='validation[:8]', 
                   shuffle_files=False)
test_ds = tfds.as_numpy(test_ds)

2024-02-23 19:11:00.677931: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-23 19:11:00.679503: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-23 19:11:00.679916: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-23 19:11:00.680480: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operation

### Model

### Training

### Testing